<a href="https://colab.research.google.com/github/EmilisEm/gmm/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Author LSP: 2213748

#### Author: Emilis Kleinas

#### Variant: Classification model with `Printer`, `Cat` and `Tiger` classes

#### What does the program do

1. Downloads images for the specified classes from `OpenImages`.
2. Initiates a clasification model.
3. Trains the defined model.
4. Calculates statistics.


Needed imports

In [1]:
!pip install -q fiftyone albumentations torchmetrics

import fiftyone.zoo as foz
import fiftyone as fo
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import matplotlib.pyplot as plt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## 1. Mount google drive to access and store images

In [2]:
from google.colab import drive
drive.mount("/content/drive")

# Define base path
DRIVE_BASE = "/content/drive/MyDrive/lab3_segmentation"
os.makedirs(DRIVE_BASE, exist_ok=True)

# Output folder
SAVE_DIR = os.path.join(DRIVE_BASE, "saved_predictions")
os.makedirs(SAVE_DIR, exist_ok=True)

Mounted at /content/drive


In [4]:
# The `data_dir` variable specifies the base directory to which the images will be saves.
# The images are stored as follows `{data_dir}/{class_name}/images/*.jpg`
# Where `class_name` is the name of the class being processed (e.g. "cow") in lowercase
data_dir = DRIVE_BASE + "/MyDrive/openimages"
number_of_images = 100
classes = ["Cat", "Tiger", "Printer"]

Download images for data classes (Optional if images already exist)

In [5]:
# Use classes available in Open Images V6

train_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    label_types=["segmentations"],
    classes=classes,
    max_samples=500,
    only_matching=True,
)

val_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="validation",
    label_types=["segmentations"],
    classes=classes,
    max_samples=100,
    only_matching=True,
)

test_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="test",
    label_types=["segmentations"],
    classes=classes,
    max_samples=100,
    only_matching=True,
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v6/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v6/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [21.0s elapsed, 0s remaining, 245.1Mb/s]      


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [21.0s elapsed, 0s remaining, 245.1Mb/s]      


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v6/train/metadata/classes.csv'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v6' split 'test'


 100% |█████████████████| 100/100 [5.2s elapsed, 0s remaining, 22.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [5.2s elapsed, 0s remaining, 22.2 samples/s]      


Dataset 'open-images-v6-test-100' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v6-test-100' created


## 2. Initiate a segmentation model

### Define custom dataset

In [6]:
class Lab3Dataset(Dataset):
    def __init__(self, fiftyone_dataset, transforms=None):
        self.samples = list(fiftyone_dataset)
        self.transforms = transforms
        self.class_to_idx = {"Cat": 1, "Tiger": 2, "Printer": 3}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample.filepath).convert("RGB")
        W, H = image.size
        mask = np.zeros((H, W), dtype=np.uint8)

        detections = sample.ground_truth.detections if sample.ground_truth else []
        for det in detections:
            if det.label not in self.class_to_idx or det.mask is None:
                continue
            inst_mask = np.array(det.mask, dtype=np.uint8)
            if inst_mask.shape != (H, W):
                inst_mask = np.array(Image.fromarray(inst_mask).resize((W, H), resample=Image.NEAREST))
            class_id = self.class_to_idx[det.label]
            mask[inst_mask > 0] = class_id

        if self.transforms:
            aug = self.transforms(image=np.array(image), mask=mask)
            return aug["image"], aug["mask"].long()

        return torch.tensor(image), torch.tensor(mask)

### Define transfroms

In [7]:
val_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0,0,0], std=[1,1,1]),
    ToTensorV2()
])

### Initiate datasets and dataloader

In [8]:
train_set = Lab3Dataset(train_dataset, transforms=val_transform)
val_set = Lab3Dataset(val_dataset, transforms=val_transform)
test_set = Lab3Dataset(test_dataset, transforms=val_transform)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)


### Define model

In [9]:
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        #Down
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        #Up
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)


Prepare model

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 1e-4
model = UNET(in_channels=3, out_channels=4).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()



<ipython-input-10-40a390b4ee8e>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


## Train model

In [11]:
from tqdm import tqdm
import torch.optim as optium

NUM_EPOCHS = 1

train_losses = []
val_losses = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = []
    loop = tqdm(train_loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE)

        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss.append(loss.item())
        loop.set_postfix(loss=loss.item())

    model.eval()
    val_loss = []

    with torch.no_grad():
        val_loop = tqdm(val_loader)
        for val_batch_idx, (val_data, val_targets) in enumerate(val_loop):
            val_data = val_data.to(device=DEVICE)
            val_targets = val_targets.to(device=DEVICE)

            with torch.cuda.amp.autocast():
                val_predictions = model(val_data)
                loss = loss_fn(val_predictions, val_targets)

            val_loss.append(loss.item())
            val_loop.set_postfix(loss=loss.item())

    avg_train_loss = sum(train_loss) / len(train_loss)
    avg_val_loss = sum(val_loss) / len(val_loss)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")



  0%|          | 0/32 [00:00<?, ?it/s]<ipython-input-11-abac5060ff35>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-11-abac5060ff35>:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 7/7 [03:36<00:00, 30.87s/it, loss=1.16]


Epoch [1/11], Train Loss: 1.1546, Val Loss: 1.2158


  0%|          | 0/32 [01:52<?, ?it/s]


KeyboardInterrupt: 

### Save predictions

In [12]:
def save_predictions(loader, model, device, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    model.eval()
    with torch.no_grad():
        for idx, (data, targets) in enumerate(loader):
            data = data.to(device)
            preds = torch.argmax(model(data), dim=1)
            for i in range(data.shape[0]):
                pred_mask = preds[i].cpu().numpy().astype(np.uint8)
                gt_mask = targets[i].cpu().numpy().astype(np.uint8)
                Image.fromarray(pred_mask * 80).save(os.path.join(save_dir, f"pred_{idx}_{i}.png"))
                Image.fromarray(gt_mask * 80).save(os.path.join(save_dir, f"gt_{idx}_{i}.png"))

save_predictions(test_loader, model, DEVICE, save_dir=SAVE_DIR)


KeyboardInterrupt: 

## Evaluate model

In [ ]:
from torchmetrics.classification import MulticlassF1Score

def dice_score(preds, targets, num_classes, epsilon=1e-6):
    dices = []
    preds = preds.flatten()
    targets = targets.flatten()
    for cls in range(num_classes):
        pred_inds = preds == cls
        target_inds = targets == cls
        intersection = (pred_inds & target_inds).sum().float()
        total = pred_inds.sum().float() + target_inds.sum().float()
        dice = (2 * intersection + epsilon) / (total + epsilon)
        dices.append(dice.item())
    return dices

NUM_CLASSES = 4
IGNORE_INDEX = 0
class_names = ["Background", "Printer", "Tiger", "Cat"]

f1_micro = MulticlassF1Score(num_classes=NUM_CLASSES, average="micro", ignore_index=IGNORE_INDEX).to(DEVICE)
f1_macro = MulticlassF1Score(num_classes=NUM_CLASSES, average="macro", ignore_index=IGNORE_INDEX).to(DEVICE)
f1_per_class = MulticlassF1Score(num_classes=NUM_CLASSES, average=None, ignore_index=IGNORE_INDEX).to(DEVICE)

model.eval()
dice_scores_all = []

with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)
        preds = torch.argmax(model(data), dim=1)
        f1_micro.update(preds, targets)
        f1_macro.update(preds, targets)
        f1_per_class.update(preds, targets)
        dice_scores_all.append(dice_score(preds, targets, NUM_CLASSES))

print(f"\nMicro F1: {f1_micro.compute().item():.4f}")
print(f"Macro F1: {f1_macro.compute().item():.4f}")
dice_avg = np.mean(dice_scores_all, axis=0)

for i, name in enumerate(class_names):
    if i == IGNORE_INDEX: continue
    print(f"{name} - F1: {f1_per_class.compute()[i]:.4f}, Dice: {dice_avg[i]:.4f}")
